In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import bs4 as bs4
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC

In [2]:
driver = webdriver.Edge()
driver.get('https://spsepchtr.mx.toyota.co.jp/kN0k/V2/servlet/jp.co.toyota.pv2y580.servlet.CCV2Y580')
user_input = driver.find_element(By.XPATH,'/html/body/div/div[2]/div[2]/div/div/form/div[2]/div[1]/table/tbody/tr[1]/td[2]/input')
user_input.send_keys('12062B5@97221')
pwd_input = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div/form/div[2]/div[1]/table/tbody/tr[2]/td[2]/input')
pwd_input.send_keys('Gpauto555#')
pwd_input.send_keys(Keys.ENTER)

In [ ]:
confirm_login_button = driver.find_element(By.XPATH,'/html/body/div/div[2]/div/div[2]/input[1]')
confirm_login_button.click()

In [3]:
# Wait for the new window/tab and then switch to it
window_handles = driver.window_handles
driver.switch_to.window(window_handles[-1])

In [4]:
#Get into the menu to search parts.
iframe = driver.find_element(By.XPATH,'/html/body/div/iframe')
driver.switch_to.frame(iframe)
partSearch_button = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[3]/div[1]/fieldset/button[2]')
partSearch_button.click()

In [ ]:
#Get into iframe that contain input box
driver.switch_to.default_content()
iframe = driver.find_element(By.XPATH,'/html/body/div[2]/iframe')
driver.switch_to.frame(iframe)

<font color='lightgreen'>Part to be lookup</font>

In [6]:
partlist = pd.read_csv(r'/Users/krichnorakhun/Documents/GP/dataResult/replyWithFormat_ml.csv')

In [7]:
partlist = partlist.loc[partlist['Brand']=='TOYOTA'][['Code']]

In [8]:
partlist['Code'].apply(lambda x : len(str(x).strip())).value_counts()

Code
10    39144
12     1854
11      404
7        41
13       11
14        6
9         2
6         1
40        1
Name: count, dtype: int64

In [9]:
partlist = partlist.loc[partlist['Code'].apply(lambda x : len(str(x).strip())) == 10].drop_duplicates()

In [10]:
partlist = partlist.loc[partlist['Code'].apply(lambda x : len(str(x).strip())) == 12]

In [11]:
partlist = partlist.drop_duplicates()

In [12]:
partlist = partlist.reset_index().drop(['index'],axis=1)

In [9]:
GPInventory = pd.read_excel(r'/Users/krichnorakhun/Documents/GP/inventory/GP.xlsx')
GPInventory = GPInventory.loc[GPInventory['ItemGroup'].apply(lambda x : str(x).find('TOYOTA'))>=0][['ItemCode']]

In [10]:
GPInventory = GPInventory.loc[GPInventory['ItemCode'].apply(lambda x : len(str(x).strip()))==14].drop_duplicates()

In [11]:

GPInventory['ItemCode'] = GPInventory['ItemCode'].apply(lambda x  : str(x).split('-')[1].strip() )
GPInventory.columns = ['Code']

In [12]:
GPInventory = GPInventory.reset_index().drop(['index'],axis=1)

In [13]:
GPInventory

,Code
0,041110E042
1,0411130544
2,04152YZZA1
3,04152YZZA5
4,041110C136
...,...
4139,PZ0460KB44
4140,PZ0460KS65
4141,PZT018702L
4142,PZT018742L


<font color='yellow'>Looping get data </font>

In [15]:
def clickInput(x):
    #Get input box
    part_1 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[1]')
    part_2 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[2]')
    part_3 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[3]')
    part_1.send_keys(x[:6])
    part_2.send_keys(x[6:11])
    #part_3.send_keys(x[11:])
    part_3.send_keys(Keys.ENTER)
    Run_Botton = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[3]/div[2]/button[10]')
    Run_Botton.click()
    return

In [16]:
def getResult_1(x,output):
    tempData = pd.DataFrame()
    soup = bs4.BeautifulSoup(x)
    tableInSoup = soup.find_all('table',{'class':'excelLikeList'})
    tempData.loc[0, 'PartID'] = tableInSoup[0].find_all('div', { 'class':'side'})[0].text
    tempData.loc[0, 'Part Description'] = tableInSoup[0].find_all('div', { 'class':'side'})[1].text
    output = pd.concat([output,tempData], ignore_index=True)
    return(output)

In [17]:
def clickNewInput():
    newFind = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[3]/div[2]/button[1]')
    newFind.click()


In [18]:
def getResult_2(data,output):
    soup = bs4.BeautifulSoup(data)
    tableInSoup = soup.find_all('table',{'class':'excelLikeList'})
    tempData = pd.DataFrame()
    for x in tableInSoup[1].find_all('tr'):
        tempData.loc[0,'PartID'] = x.find_all('td')[1].text
        tempData.loc[0,'PNC'] = x.find_all('td')[2].text
        tempData.loc[0,'Count'] = x.find_all('td')[3].text
        tempData.loc[0,'Catalogue'] = x.find_all('td')[4].text
        tempData.loc[0,'Model'] = x.find_all('td')[5].text
        tempData.loc[0,'Fr-To'] = x.find_all('td')[6].text
        tempData.loc[0,'Model(Details)'] = x.find_all('td')[7].text
        output = pd.concat([output, tempData], ignore_index=True)
    return(output)
    #print(x.find_all('td')[1].text)

In [19]:
def wait_for_page_load(driver, timeout=30):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: d.execute_script('return document.readyState') == 'complete'
        )
    except TimeoutException:
        print("Timed out waiting for page to load")

In [7]:
MainTable = pd.DataFrame()
SecondTable = pd.DataFrame()

In [ ]:
for index, row in GPInventory.iloc[568:].iterrows():
    clickInput(row['Code'].strip())
    wait = WebDriverWait(driver, 60) 
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)

    try:
        wait.until(EC.visibility_of_element_located((By.ID, 'eltColF10_divListSelectHinban4')))
        data = driver.page_source
        soup = bs4.BeautifulSoup(data)
        if soup.find_all('div',{'id':'eltColF10_divListSelectHinban4'})[0].text != 'ไม่มีใช้งาน':
            wait.until(EC.visibility_of_element_located((By.ID, 'eltRowF10_divListSearch1')))
       
        time.sleep(1)
        data = driver.page_source
        MainTable = getResult_1(data,MainTable)
        SecondTable = getResult_2(data,SecondTable)
        print(index)
        
    except TimeoutException:
        print("Timeout occurred! Performing alternative steps.")

    finally:
        clickNewInput()

In [889]:
clickNewInput()

In [902]:
MainTable = MainTable.drop_duplicates()

In [905]:
MainTable = MainTable.loc[MainTable['Part Description']!='']

In [906]:
MainTable.to_excel(r'/Users/krichnorakhun/Documents/GP/inventory/EPC_Main_from_Inventory_full.xlsx', index=False)

In [909]:
SecondTable = SecondTable.drop_duplicates()

In [910]:
SecondTable.to_excel(r'/Users/krichnorakhun/Documents/GP/inventory/EPC_Secondary_from_Inventory_full.xlsx', index=False)

In [13]:
pd.read_excel(r'/Users/krichnorakhun/Documents/GP/inventory/EPC_Secondary_from_Inventory_full.xlsx')

,PartID,PNC,Count,Catalogue,Model,Fr-To,Model(Details)
0,04111-30544,04111,01,711350,HIACE,201005- 201309,2KDFTV..KDH222..TH
1,04112-0L191,04112,01,6713A0,FORTUNER,201410- 201611,"2KDFTV..KUN50,60..IDN,MA,PHL,SA,VN*130"
2,04112-0L191,04112,01,672380,HILUX,201410- 201601,"2KDFTV..KUN25..IDN,MA2KDFTV..KUN35..DLX..PHL2K..."
3,04112-0L191,04112,01,672390,HILUX,201505- 201707,"2KDFTV..KUN125,135..HDCR"
4,04226-0L010,04226,01,611260,INNOVA/KIJANG INNOVA,200507- 200708,2KDFTV..KUN40..IND*163 OR *184
...,...,...,...,...,...,...,...
151463,99366-H1700,16361A,01,156360,COROLLA/ALTIS (ASIA),200903- 201105,"1ZZFE,3ZZFE..ZZE14#..IDN,IND,MA,PHL,PKN,TAIW,T..."
151464,99366-H1700,16361A,01,156360,COROLLA/ALTIS (ASIA),201008- 201311,3ZZFE..ZZE141
151465,99366-H1700,16361A,01,513120,ISIS,200903- 200909,1ZZFE..ZNM10
151466,99367-J1960,16361A,01,283320,CAMRY/HYBRID (ASIA),201112- 201504,1AZFE..ACV51
